## Extract

In [1]:
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
url = 'https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes'

In [3]:
# function to read table in website using pandas
def scrape(url):
    logging.info(f' scrapping dari url: {url}')
    return pd.read_html(url, header=None)

In [4]:
# scrapping second table from website
df = scrape(url)[1]

INFO:root: scrapping dari url: https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes
INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [30]:
# preview data
df.head()

,No.,Nama,Kekayaan bersih (USD),Usia,Kebangsaan,Sumber kekayaan
0,NaN,Jeff Bezos,$177 miliar,57,Amerika Serikat,Amazon
1,NaN,Elon Musk,$151 miliar,49,Amerika Serikat,"Tesla, SpaceX"
2,NaN,Bernard Arnault & keluarga,$150 miliar,72,Prancis,LVMH
3,NaN,Bill Gates,$124 miliar,65,Amerika Serikat,Microsoft
4,NaN,Mark Zuckerberg,$97 miliar,36,Amerika Serikat,Meta Platforms


## Transform

In [6]:
# checking data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   No.                    0 non-null      float64
 1   Nama                   10 non-null     object 
 2   Kekayaan bersih (USD)  10 non-null     object 
 3   Usia                   10 non-null     int64  
 4   Kebangsaan             10 non-null     object 
 5   Sumber kekayaan        10 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 608.0+ bytes


In [7]:
# import library to replace substrings using regex
import re

In [8]:
# function to check if nett wealth ends with 'miliar'
def is_money_miliar(string_money):
    return string_money.lower().endswith('miliar')

# function to replace unused substrings in nett wealth column
def transform_money_format(string_money):
    half_clean_string = string_money.lower().replace(',','.').replace(' ','')
    return re.sub(r'[\$miliar$]',' ',half_clean_string)

# function that combine two functions above
def transform(df, tahun):
    logging.info('transforming...')
    
    # columns mapping to rename
    columns_mapping = {
        'No.':'nomor_urut',
        'Nama':'nama',
        'Usia':'usia',
        'Kebangsaan':'kebangsaan',
        'Sumber kekayaan':'sumber_kekayaan',
        'Kekayaan bersih (USD)':'kekayaan_bersih_usd'
    }
    
    # rename columns
    renamed_df = df.rename(columns = columns_mapping)
    
    # add year column
    renamed_df['tahun'] = tahun
    
    # add 'perusahaan' column
    renamed_df['perusahaan'] = renamed_df['sumber_kekayaan']
    
    # insert ranking number
    renamed_df['nomor_urut'] = list(range(1,len(df)+1))
    
    # clean wealth column and change data type to float
    renamed_df['kekayaan_bersih_usd'] = renamed_df['kekayaan_bersih_usd'].apply(
        lambda x: float(transform_money_format(x)) * 1000 if is_money_miliar(x) else float(transform_money_format(x))
    )
    
    logging.info('done')
    
    # return cleaned data and ordering column
    return renamed_df[['nama','nomor_urut','kekayaan_bersih_usd','perusahaan','usia','kebangsaan','sumber_kekayaan','tahun']]

In [9]:
# clean data
df_2021 = transform(df, 2021)

INFO:root:transforming...
INFO:root:done


In [10]:
# preview table
df_2021

,nama,nomor_urut,kekayaan_bersih_usd,perusahaan,usia,kebangsaan,sumber_kekayaan,tahun
0,Jeff Bezos,1,177000.0,Amazon,57,Amerika Serikat,Amazon,2021
1,Elon Musk,2,151000.0,"Tesla, SpaceX",49,Amerika Serikat,"Tesla, SpaceX",2021
2,Bernard Arnault & keluarga,3,150000.0,LVMH,72,Prancis,LVMH,2021
3,Bill Gates,4,124000.0,Microsoft,65,Amerika Serikat,Microsoft,2021
4,Mark Zuckerberg,5,97000.0,Meta Platforms,36,Amerika Serikat,Meta Platforms,2021
5,Warren Buffett,6,96000.0,Berkshire Hathaway,90,Amerika Serikat,Berkshire Hathaway,2021
6,Larry Ellison,7,93000.0,Oracle Corporation,76,Amerika Serikat,Oracle Corporation,2021
7,Larry Page,8,91500.0,Alphabet Inc.,48,Amerika Serikat,Alphabet Inc.,2021
8,Sergey Brin,9,89000.0,Alphabet Inc.,47,Amerika Serikat,Alphabet Inc.,2021
9,Mukesh Ambani,10,84500.0,Reliance Industries,63,India,Reliance Industries,2021


## Load

In [19]:
# import library to load dataframe to database
from sqlalchemy import create_engine

In [25]:
RDBMS = 'postgresql'
DRIVER = 'psycopg2'
DB_NAME = 'postgres'
DB_USER = 'user1'
DB_PASS = 'user1'
DB_HOST = '104.197.148.144'
DB_PORT = '5432'

# connection string to connect to database
CONNECTION_STRING = f'{RDBMS}+{DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# table name to be created
TABLE_NAME = 'KalisRifoIrwandi_orang_terkaya_forbes'

In [26]:
# function to create new table and replace if it already exist
def write_to_db(df, connection_string, nama_tabel):
    logging.info('connecting to database...')
    engine = create_engine(connection_string)
    logging.info(f'create {nama_tabel} table...')
    df.to_sql(nama_tabel, con=engine, index=False, if_exists='replace')
    logging.info('done')

In [27]:
# create new table to database
write_to_db(df_2021,CONNECTION_STRING,TABLE_NAME)

INFO:root:connecting to database...
INFO:root:create KalisRifoIrwandi_orang_terkaya_forbes table...
INFO:root:done
